In [ ]:
%pip install openai
%pip install langchain
%pip install python-dotenv
%pip install chromadb
%pip install pypdf
%pip install Flask
%pip install tiktoken

import os
import traceback
from dotenv import load_dotenv
from langchain import OpenAI
from langchain import PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.embeddings import OpenAIEmbeddings
from flask import Flask, send_file
from flask import request,abort, render_template, jsonify

os.environ['OPENAI_API_KEY'] = 'OPENAI_API_KEY'

app = Flask(__name__)

load_dotenv()

prompt_template = """

Use the following context to answer the question. 

Say "Sorry I don't know the answer" if you don't know the answer. 

{context}

Question: {question}
"""

prompt = PromptTemplate.from_template(prompt_template)

def get_chunks(data):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1000, chunk_overlap=50)
    
    chunks = splitter.split_documents(data)

    return chunks

def get_db_retriever(chunks):
    embeddings = OpenAIEmbeddings()
    db = Chroma.from_documents(chunks,embeddings)

    return db.as_retriever()

def load_data():
    print("Loading data from PDF document")
    pdf_loader = PyPDFLoader("file_name.pdf")
    data = pdf_loader.load()

    return data

def retrieval_qa(q):
    
    llm = OpenAI()

    data = load_data()

    chunks = get_chunks(data)

    db_retriever = get_db_retriever(chunks)

    retrievalQA = RetrievalQA.from_chain_type(llm, 
        chain_type="stuff", 
        retriever=db_retriever,
        chain_type_kwargs={"prompt": prompt})

    return retrievalQA({"query": q})

def ask(question):
    return retrieval_qa(question)

@app.route("/")
def index():
    try:
        return send_file(os.path.join('templates', 'chat-interface.html'))
    except Exception as e:
        print(f"Error sending HTML file: {e}")
        abort(500, description="Internal Server Error")

@app.route("/call", methods=["POST"])
def call():
    try:
        req = request.json
        print(f"Received request: {req}")
        question = req.get('question')

        response = ask(question)
        print(f"Response: {response}")

        # Using jsonify to convert the response to JSON
        return jsonify({'answer': response})
    except Exception as e:
        print(f"Error in call: {e}")
        abort(500, description="Internal Server Error")

if __name__== '__main__':
    app.run(host="0.0.0.0",port=3099)